In [1]:
import xml.etree.ElementTree as ET

In [2]:
# source from https://stackoverflow.com/a/1206856/8865471
from xml.dom import minidom
def prettify(elem):
    """Return a pretty-printed XML string for the Element.
    """
    rough_string = ET.tostring(elem, 'utf-8')
    reparsed = minidom.parseString(rough_string)
    return reparsed.toprettyxml(indent="\t")

In [3]:
p0="100000.0"

In [4]:
f = open('thermo.txt','r')
lines = f.readlines()
f.close()

In [5]:
content = []
for line in lines:
    l = line.strip().split()
    content.append(l)

In [6]:
species = ['O','O2','H','H2','OH','H2O','HO2','H2O2']

In [7]:
species_i = []
first = [c[0] for c in content if c]
for i, f in enumerate(first):
    if f == 'THERMO':
        thermo_i = i+1
    for j, s in enumerate(species):
        if f == s:
            species_i.append(i)
            del species[j]
# species_i

In [8]:
temperature = content[thermo_i]
temperature

['300.000', '1000.000', '5000.000']

In [9]:
coeffs = []
for i in species_i:
    numbers = content[i+1][:-1] + content[i+2][:-1] + content[i+3][:-1]
    coeffs.append(numbers)
# coeffs

In [10]:
total_list = []
for lst in coeffs:
    new_list = []
    for i, n in enumerate(lst):
        if len(n) == 29:
            lst[i] = n[:14] + ' ' + n[14:]
        elif len(n) == 30:
            lst[i] = n[:15] + ' ' + n[15:]
        elif len(n) == 44:
            lst[i] = n[:14] + ' ' + n[14:29] + ' ' + n[29:]
        new = lst[i].strip().split()
        new_list.append(new)
    concat_list = [j for i in new_list for j in i]
    total_list.append(concat_list)
# total_list

In [11]:
species = ['O','O2','H','H2','OH','H2O','HO2','H2O2']
d1 = {}
for i, t in enumerate(total_list):
    d1[species[i]] = {}
    d1[species[i]]['high'] = t[:7]
    d1[species[i]]['low'] = t[7:]
# d1

In [12]:
root = ET.Element("ctml")
doc = ET.SubElement(root, "phase", id="gri30")
ET.SubElement(doc, "speciesArray", datasrc="#species_data").text = 'O O2 H H2 OH H2O HO2 H2O2'

In [13]:
doc2 = ET.SubElement(root, "speciesData", id="species_data")
for specie in species:
    comment = ET.Comment("species " + specie)
    s = ET.SubElement(doc2, "species", name=specie)
    s.append(comment)
    s_thermo = ET.SubElement(s, "thermo")
    s_NASA_high = ET.SubElement(s_thermo, "NASA", TMAX=temperature[2], TMIN=temperature[1], P0=p0)
    s_fa_high = ET.SubElement(s_NASA_high, "floatArray", name="coeffs", size="7").text = ','.join(d1[specie]['high'])
    s_NASA_low = ET.SubElement(s_thermo, "NASA", TMAX=temperature[1], TMIN=temperature[0], P0=p0)
    s_fa_low = ET.SubElement(s_NASA_low, "floatArray", name="coeffs", size="7").text = ','.join(d1[specie]['low'])

In [14]:
tree = ET.ElementTree(root)
tree.write("thermo.xml")
with open("thermo.xml", 'w') as f:
    f.write(prettify(root))